<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/2_GPT_multiclass_batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP final - reconocimiento de discursos discriminatorios en Twitter

## Clasificacion multietiquetas con GPT 4 mini (sin finetunning) por batch


**Natalia Dedandi**





La documentación de la API se encuentra aqui:

https://platform.openai.com/docs/api-reference/introduction



In [ ]:
pip install openai

In [ ]:
!pip install datasets seaborn


In [ ]:
import json
from openai import OpenAI
import os
from google.colab import userdata
#seteo la KEY
os.environ['OPENAI_API_KEY'] = userdata.get('openIA_key')




In [ ]:
#creo el cliente OPENAI con mi usuario y proyecto
client = OpenAI(
  organization='org-1uHjwiaB3OlPzoxfVzhqOSzs',
  project='proj_2fII7izwVGgYaERNNKhhMx4l',
)

Debo cargar mi dataset de TEST y pasarle un archivo BATCH para que lo procese

Usare este mensaje para cada Tweet:

"role": "system", "content": "You are trained to analyze and detect the sentiment of the given text. Only hateful or not hateful is allowed. The answer should be in spanish"

In [ ]:
from datasets import load_dataset
import pandas as pd

ds1 = load_dataset("piuba-bigdata/contextualized_hate_speech")


In [ ]:
ds1

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 36420
    })
    test: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 11343
    })
    dev: Dataset({
        features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
        num_rows: 9106
    })
})

In [ ]:
#tomo una muestra de 100 TW random
df_test = pd.DataFrame(ds1['test'])
df_test

,id,title,text,context_tweet,HATEFUL,body,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL
0,397703,Mora Godoy cierra su escuela de tango y remata...,@usuario Ojalá se funda y cague de hambre,Mora Godoy cierra su escuela de tango y remata...,0,Puede marcar con justeza la fecha de inicio de...,0,0,0,0,0,0,0,0,0
1,397704,Mora Godoy cierra su escuela de tango y remata...,@usuario Faltó que se enganchó al hijo de sofo...,Mora Godoy cierra su escuela de tango y remata...,1,Puede marcar con justeza la fecha de inicio de...,0,1,0,0,0,0,0,0,0
2,397705,Mora Godoy cierra su escuela de tango y remata...,"@usuario y bue, que le vaya a llorar a Cristin...",Mora Godoy cierra su escuela de tango y remata...,0,Puede marcar con justeza la fecha de inicio de...,0,0,0,0,0,0,0,0,0
3,397706,Mora Godoy cierra su escuela de tango y remata...,@usuario @usuario Jodete. Macri te hizo conoce...,Mora Godoy cierra su escuela de tango y remata...,0,Puede marcar con justeza la fecha de inicio de...,0,0,0,0,0,0,0,0,0
4,397707,Mora Godoy cierra su escuela de tango y remata...,"@usuario Que se joda, es su gobierno el que el...",Mora Godoy cierra su escuela de tango y remata...,0,Puede marcar con justeza la fecha de inicio de...,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11338,361071,Nahir Galarza sumó a una abogada feminista a s...,@usuario Con esa varita q no rompe un plato ni...,Nahir Galarza sumó a una abogada feminista a s...,0,"La defensa de Nahir Galarza, condenada el 3 de...",0,0,0,0,0,0,0,0,0
11339,361072,Nahir Galarza sumó a una abogada feminista a s...,@usuario No las entiendo a estas pelotudas. So...,Nahir Galarza sumó a una abogada feminista a s...,1,"La defensa de Nahir Galarza, condenada el 3 de...",0,1,0,0,0,0,0,0,0
11340,361073,Nahir Galarza sumó a una abogada feminista a s...,@usuario Lo mató por qué es macho!,Nahir Galarza sumó a una abogada feminista a s...,0,"La defensa de Nahir Galarza, condenada el 3 de...",0,0,0,0,0,0,0,0,0
11341,361074,Nahir Galarza sumó a una abogada feminista a s...,@usuario Cuando sos asesina tenés que hacerte ...,Nahir Galarza sumó a una abogada feminista a s...,0,"La defensa de Nahir Galarza, condenada el 3 de...",0,0,0,0,0,0,0,0,0


Documentacion para el trabajo en batch:
https://platform.openai.com/docs/guides/batch/getting-started


Se prepara el formato completo del archivo para batch con el siguiente formato:

{"custom_id": "request-1", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-3.5-turbo-0125", "messages": [{"role": "system", "content": "You are a helpful assistant."},{"role": "user", "content": "Hello world!"}],"max_tokens": 1000}}


Al rol System se le envía la siguiente consigna:

You must classify a text with one or more of the following labels: Label WOMEN discrimination against women, label LGBTI gender discrimination, specifically against gays, lesbians or transsexuals,label RACISM is racism, or discrimination against people according to their skin color,label
CLASS is discrimination for being poor or living in a marginal or poor place, label POLITICS is hatred of politicians, label DISABLED is discrimination against people with disabilities, label APPEARENCE is discrimination against people because of their physical appearance, specifically fatphobia, label CRIMINAL is discrimination against people because of a criminal record or links to criminality and label CALLS is hateful text that incites violence
If the text is not hatefull indicate label NOT HATEFUL

In [ ]:
def create_batch_multiple(df):
  # Specify the columns
  id_column = 'id'
  text_column = 'text'

  # Convert the dataframe into a list of dictionaries with custom_id and messages
  batch = [
      {
          'custom_id': str(row[id_column]),
          'method': 'POST',
          'url': '/v1/chat/completions',
          'body':{
              'model': 'gpt-4o-mini',
              'messages': [
                  {'role': 'system', 'content': 'You are trained to analyze and detect the sentiment of the given text.You must classify a text with one or more of the following labels: WOMEN when there is discrimination against women, LGBTI when there is discrimination against gays, lesbians or transsexuals, RACISM when there is discrimination against people according to their skin color, CLASS  when there is discrimination for being poor or living in a marginal or poor place, POLITICS is hatred of politicians, DISABLED when there is discrimination against people with disabilities, APPEARENCE when there is discrimination against people because of their physical appearance, specifically fatphobia, CRIMINAL when there is discrimination against people because of a criminal record or links to criminality and  CALLS is hateful text that incites violence. If the text is not hateful indicate NOT HATEFUL'},
                  {'role': 'user', 'content': row[text_column]}
                  ],
              'max_tokens': 20,  # Limit response to a single word
              'temperature': 0  # Keep response consistent
          }
      }
      for _, row in df.iterrows()
  ]
  return(batch)

Como no puedo hacer un solo batch porque se excede el limite voy a probar con trabajos de mitad

In [ ]:
size_df=len(df_test)
size_df=int(size/2)
size_df

2836

In [118]:
import numpy as np
def split_df(df):
  if len(df) % 2 != 0:  # Handling `df` with `odd` number of rows
    df = df.iloc[:-1, :]
    df1, df2 =  np.array_split(df, 2)
    return df1, df2

In [119]:
df1, df2 = split_df(df_test)

In [122]:
len(df1)

5671

In [124]:
batch1=create_batch_multiple(df1)
batch1[0]

{'custom_id': '397703',
 'method': 'POST',
 'url': '/v1/chat/completions',
 'body': {'model': 'gpt-4o-mini',
  'messages': [{'role': 'system',
    'content': 'You are trained to analyze and detect the sentiment of the given text.You must classify a text with one or more of the following labels: WOMEN when there is discrimination against women, LGBTI when there is discrimination against gays, lesbians or transsexuals, RACISM when there is discrimination against people according to their skin color, CLASS  when there is discrimination for being poor or living in a marginal or poor place, POLITICS is hatred of politicians, DISABLED when there is discrimination against people with disabilities, APPEARENCE when there is discrimination against people because of their physical appearance, specifically fatphobia, CRIMINAL when there is discrimination against people because of a criminal record or links to criminality and  CALLS is hateful text that incites violence. If the text is not hateful 

In [125]:
len(batch1)

5671

In [126]:
batch2=create_batch_multiple(df2)
batch2[0]

{'custom_id': '398942',
 'method': 'POST',
 'url': '/v1/chat/completions',
 'body': {'model': 'gpt-4o-mini',
  'messages': [{'role': 'system',
    'content': 'You are trained to analyze and detect the sentiment of the given text.You must classify a text with one or more of the following labels: WOMEN when there is discrimination against women, LGBTI when there is discrimination against gays, lesbians or transsexuals, RACISM when there is discrimination against people according to their skin color, CLASS  when there is discrimination for being poor or living in a marginal or poor place, POLITICS is hatred of politicians, DISABLED when there is discrimination against people with disabilities, APPEARENCE when there is discrimination against people because of their physical appearance, specifically fatphobia, CRIMINAL when there is discrimination against people because of a criminal record or links to criminality and  CALLS is hateful text that incites violence. If the text is not hateful 

In [127]:
len(batch2)

5671

Creo un batch separado por cada uno porque no soporta archivos tan grandes

In [128]:
# Save the batch to a .jsonl file
with open('batch_gpt_multiclass_1.jsonl', 'w') as f:
    for entry in batch1:
        json.dump(entry, f)
        f.write('\n')

print("Batch file created successfully in .jsonl format!")

Batch file created successfully in .jsonl format!


In [129]:
batch_input_file = client.files.create(
  file=open("batch_gpt_multiclass_1.jsonl", "rb"),
  purpose="batch"
)

In [130]:
batch_input_file

FileObject(id='file-e91dR30ggfN2v3L7E32F3aGl', bytes=6696011, created_at=1721950345, filename='batch_gpt_multiclass_1.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [131]:
batch_input_file_id = 'file-e91dR30ggfN2v3L7E32F3aGl'

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "nightly eval job"
    }
)

Batch(id='batch_lbqgkIXz96h4G82byrWsKiLf', completion_window='24h', created_at=1721950358, endpoint='/v1/chat/completions', input_file_id='file-e91dR30ggfN2v3L7E32F3aGl', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1722036758, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [132]:
client.batches.retrieve('batch_lbqgkIXz96h4G82byrWsKiLf')

Batch(id='batch_lbqgkIXz96h4G82byrWsKiLf', completion_window='24h', created_at=1721950358, endpoint='/v1/chat/completions', input_file_id='file-e91dR30ggfN2v3L7E32F3aGl', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1722036758, failed_at=None, finalizing_at=None, in_progress_at=1721950361, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=5671))

In [ ]:
batch_NQ9RgzBbLyqfmLOnoBYq0OPG

Ahora creo el segundo batch

In [133]:
with open('batch_gpt_multiclass_2.jsonl', 'w') as f:
    for entry in batch2:
        json.dump(entry, f)
        f.write('\n')

print("Batch file created successfully in .jsonl format!")

Batch file created successfully in .jsonl format!


In [134]:
batch_input_file = client.files.create(
  file=open("batch_gpt_multiclass_2.jsonl", "rb"),
  purpose="batch"
)
batch_input_file

FileObject(id='file-0HJYdfaKzj7JkfW7PxmAyf7M', bytes=6682141, created_at=1721951551, filename='batch_gpt_multiclass_2.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [135]:
batch_input_file_id = 'file-0HJYdfaKzj7JkfW7PxmAyf7M'

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "nightly eval job"
    }
)

Batch(id='batch_whoRan66MYdbw5OhNxJqFHhu', completion_window='24h', created_at=1721951567, endpoint='/v1/chat/completions', input_file_id='file-0HJYdfaKzj7JkfW7PxmAyf7M', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1722037967, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [136]:
client.batches.retrieve('batch_whoRan66MYdbw5OhNxJqFHhu')

Batch(id='batch_whoRan66MYdbw5OhNxJqFHhu', completion_window='24h', created_at=1721951567, endpoint='/v1/chat/completions', input_file_id='file-0HJYdfaKzj7JkfW7PxmAyf7M', object='batch', status='in_progress', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1722037967, failed_at=None, finalizing_at=None, in_progress_at=1721951570, metadata={'description': 'nightly eval job'}, output_file_id=None, request_counts=BatchRequestCounts(completed=278, failed=0, total=5671))